# Algoritmos de optimización - Seminario<br>

Nombre y Apellidos:         Eva Maria Martin Cruz

Url: https://github.com/emmartinc/Algoritmos_de_optmizacion_Eva_Martin/ACTIVIDAD_FINAL_Seminario_Algoritmos_Definitiva.ipynb<br>

Problema:
## 1. Sesiones de doblaje <br>
                                        

Se precisa coordinar el doblaje de una película. 

Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. 

Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. 

No es posible grabar más
de 6 tomas por día. 

El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible. 

Los datos son:

Número de actores: 10

Número de tomas : 30

Actores/Tomas : https://bit.ly/36D8IuK
- 1 indica que el actor participa en la toma
- 0 en caso contrario

## ¿Cuantas posibilidades hay sin tener en cuenta las restricciones?


Respuesta

Para calcular cuántas posibilidades hay sin tener en cuenta las restricciones, necesitamos contar todas las posibles formas de asignar las 30 tomas a cualquier número de días. 
Esto va desde realizar las 30 tomas todas el mismo dia, hasta realizar una toma diaria, pasando por: rodar cada dia 15 tomas y ahcerlo en 2 días, rodar cada dia 5 tomas y hacerlo en 6 días...
todo esto representa en definitiva todos los conjuntos que podemos hacer con 30 elementos: 2^30


In [1]:
#Zona de imports - agrupamos todas las libreria necesarias arriba del todo para tenerlas importadas a lo largo del problema.
import pandas as pd
import math
import numpy as np
import random


In [3]:
# Datos del problema

# NUMERO DE TOMAS
num_tomas = 30

# NUMERO DE ACTORES
num_actores = 10

#MAXIMO NUMERO DE TOMAS POR DIA
max_tomas_por_dia = 6

# archivo Excel
archivo_excel = 'Datos problema.xlsx'

In [4]:
# Calcular 2 ^30
posibilidades_sin_restricciones = pow(2, num_tomas)

print(f"El número de posibilidades sin tener en cuenta las restricciones es: {posibilidades_sin_restricciones}")


El número de posibilidades sin tener en cuenta las restricciones es: 1073741824


Se trata de un número enorme, por lo que nos podemos dar cuenta del número masivo de combinaciones posibles sin restricciones.

Aunque realmente, para el objetivo de nuestro problema, el que no haya restricciones lo que significaria es que se pueden hacer las 30 tomas en 1 solo día con todos los actores presentes, con lo que el coste sería el mínimo posible.

Es decir, me parece resaltable darnos cuenta de que el orden de las tomas dentro del mismo dia no importa.


## ¿Cuántas posibilidades hay teniendo en cuenta todas las restricciones?

Para calcular las posibilidades teniendo en cuenta todas las restricciones,
empezamos con la primera y vamos añadiendo complejidad.

Consideramos que la primera restricción del problema es que no es posible grabar más de 6 tomas por día.

Ahí ya estariamos eliminando todos los conjuntos superiores a 6 

In [5]:
# Calculamos la combinación

combinaciones = 0
for i in range(1, 7):  # Iteramos sobre los valores de i desde 1 hasta 6
    combinaciones += math.comb(num_tomas, i)  # Sumamos la combinación de i elementos tomados max_tomas_dia a la vez

print("El número de combinaciones posibles es:", combinaciones)

El número de combinaciones posibles es: 768211


Luego, calculamos el mínimo número de días que necesitamos para encajar las tomas, ya que otra restricción del problema es que los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben por lo que nos interesa minimizar el numeros de dias de rodaje.
Tenemos que fijarnos en este número de dias que nos sale e intentar tender a él.


In [6]:
min_dias = math.ceil(num_tomas / max_tomas_por_dia)

print ("El numero minimo de dias necesario es: ", min_dias)

El numero minimo de dias necesario es:  5



Restriccion 2: Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas.

Vamos a definir la matriz de participación de los actores en cada toma. 

Esto nos va a ir marcando qué actores tenemos ese día en nuestro conjunto de combinaciones.


In [7]:

# Leer el archivo Excel en un DataFrame,
participacion_df = pd.read_excel(archivo_excel)  

# Imprimir la matriz de participación
#print("Matriz de Participación:")


#print(participacion_df)



### Modelo para el espacio de soluciones<br>
### (*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


 matrices, al principio habia pensado conjuntos, diccionario de conjuntos, pero no es necesario,
 Matriz de Participación (DataFrame de pandas)

Justificación:
Claridad y Simplicidad: Una matriz de participación donde las filas representan a los actores y las columnas representan las tomas permite una visualización clara y directa de qué actores participan en cada toma. 

Esto facilita la identificación rápida de las tomas que pueden agruparse en un mismo día.

Operaciones Eficientes: Las operaciones de acceso y modificación de elementos en una matriz son muy eficientes.

Dado que necesitamos verificar la participación de actores en diferentes tomas frecuentemente, la matriz permite realizar estas operaciones en tiempo constante  O(1).


Compatibilidad con Herramientas de Análisis: Utilizando un DataFrame de pandas, podemos aprovechar una amplia gama de funciones de análisis y manipulación de datos, lo cual es extremadamente útil para la preprocesamiento de los datos y la aplicación de algoritmos de optimización.

También es verdad que según el algoritmo que he implementado he añadido o refinado el uso de los tipos de datos.

Matriz de Participación (participation_matrix):

Tipo: numpy.ndarray
Justificación: La matriz de participación es una estructura bidimensional que indica qué actores participan en qué tomas. Usar un numpy.ndarray es eficiente tanto en términos de memoria como de operaciones de acceso y manipulación, que son necesarias para evaluar rápidamente los actores participantes en cualquier subconjunto de tomas.


Tabla de Programación Dinámica (dp):

Tipo: lista
Justificación: La tabla dp es una lista unidimensional donde dp[i] almacena el costo mínimo para grabar las primeras i tomas. Usar una lista es suficiente y eficiente para este propósito ya que solo necesitamos acceder y actualizar los costos de forma secuencial y directa.
Lista de Particiones (particiones):


Lista de Resultados (dias):

Tipo: lista de tuplas
Justificación: La lista dias almacena tuplas que representan los rangos de tomas asignadas a cada día. Este formato es conveniente para reconstruir y presentar la secuencia óptima de tomas por día. Usar una lista de tuplas facilita la manipulación y el acceso ordenado a los datos.


DataFrame de Resultados (resultado):

Tipo: pandas.DataFrame


Para la presentación final de los resultados, un pandas.DataFrame es el ideal.

Elección de las Estructuras de Datos

numpy.ndarray para participation_matrix: Esta elección se debe a la necesidad de realizar operaciones vectorizadas rápidas y eficientes al calcular la participación de los actores en múltiples tomas.

list para dp y particiones: Las listas proporcionan una estructura simple y eficiente para almacenar los resultados de la programación dinámica y las decisiones de partición.

pandas.DataFrame para resultado: El uso de un DataFrame facilita la manipulación y visualización de los resultados finales, permitiendo una presentación clara y ordenada.

Consideraciones Iniciales y Ajustes
Inicialmente, consideré usar listas para todas las estructuras de datos debido a su simplicidad y facilidad de uso. Sin embargo, al trabajar con la matriz de participación, opté por numpy.ndarray debido a sus ventajas en términos de rendimiento para operaciones matemáticas y lógicas.

En resumen, la combinación de numpy y pandas junto con listas nativas de Python proporciona un equilibrio óptimo entre eficiencia, simplicidad y capacidad de presentación para resolver y visualizar este problema de optimización.

### Según el modelo para el espacio de soluciones<br>
### (*)¿Cual es la función objetivo?

### (*)¿Es un problema de maximización o minimización?

Es un problema de minimización de costes, que teniendo en cuenta que son fijos por actor y dia, minimizar el coste de esta función es tambien minimizar el numero de dias 

### Diseña un algoritmo para resolver el problema por fuerza bruta

### Calcula la complejidad del algoritmo por fuerza bruta

Pues es que la complejidad del algoritmo por fuerza bruta es paralela al orden de las 768211 combinaciones calculadas en el apartado anterior.

Es un algoritmo que no se puede ejecutar porque nunca termina. (Se queda pillado). 



Aquí está el diseño:

### Paso 1. Generar todas las combinaciones posibles de tomas por día:

Para cada día, generar todas las combinaciones posibles de hasta 6 tomas.

### Paso 2. Calcular el costo de cada combinación:

Para cada combinación de tomas en un día, calcular cuántos actores deben desplazarse.
Sumar los costos diarios de desplazamiento para obtener el costo total de esa combinación de días.

### Paso 3. Elegir la combinación con el costo mínimo:

Comparar los costos de todas las combinaciones posibles y seleccionar la que minimice el gasto total.



# NOTA MUY IMPORTANTE

Se añaden los parámetros

max_combinaciones =  # Máximo número de combinaciones permitidas

max_permutaciones =  # Máximo número de permutaciones permitidas


DE ESTA MANERA, PODEMOS PROBAR EL FUNCIONAMIENTO DEL ALGORITMO SIN QUE SE EJECUTE INDEFINIDAMENTE.

Sabemos que no es la solución buena, pero tenemos que probar que nos funcionaria.


In [8]:
participacion_np = participacion_df.drop(columns=['Toma']).to_numpy()

# Función para calcular el costo de desplazamiento de una configuración dada
def calcular_costo(configuracion):
    costo_total = 0
    for dia in configuracion:
        actores_dia = set()
        for toma in dia:
            actores_dia.update(np.where(participacion_np[toma] == 1)[0])
        costo_total += len(actores_dia)
    return costo_total

# Generar combinaciones limitadas
def generar_combinaciones(tomas, k, limite):
    resultado = []
    combinaciones_contador = [0]  # Usamos una lista mutable para el contador

    def combinacion(inicio, camino):
        if combinaciones_contador[0] >= limite:  # Detener después de alcanzar el límite
            return
        if len(camino) == k:
            resultado.append(camino[:])
            combinaciones_contador[0] += 1
            return
        for i in range(inicio, len(tomas)):
            camino.append(tomas[i])
            combinacion(i + 1, camino)
            camino.pop()
    
    combinacion(0, [])
    return resultado

# Generar permutaciones limitadas
def generar_permutaciones(arr, k, limite):
    resultado = []
    usados = [False] * len(arr)
    permutaciones_contador = [0]  # Usamos una lista mutable para el contador

    def permutacion(camino):
        if permutaciones_contador[0] >= limite:  # Detener después de alcanzar el límite
            return
        if len(camino) == k:
            resultado.append(camino[:])
            permutaciones_contador[0] += 1
            return
        for i in range(len(arr)):
            if not usados[i]:
                usados[i] = True
                camino.append(arr[i])
                permutacion(camino)
                camino.pop()
                usados[i] = False
    
    permutacion([])
    return resultado

# Lista de tomas
tomas = list(range(30))

# Parámetros
max_combinaciones = 10  # Máximo número de combinaciones permitidas
max_permutaciones = 10  # Máximo número de permutaciones permitidas

# Generar combinaciones de tomas con límite
combinaciones = generar_combinaciones(tomas, 6, max_combinaciones)

# Generar permutaciones de combinaciones con límite
permutaciones = generar_permutaciones(combinaciones, 5, max_permutaciones)

# Encontrar la mejor configuración
mejor_costo = float('inf')
mejor_configuracion = None

for config in permutaciones:
    tomas_cubiertas = set()
    for dia in config:
        for toma in dia:
            tomas_cubiertas.add(toma)
    if len(tomas_cubiertas) == 30:  # Asegurarse de cubrir todas las tomas
        costo_actual = calcular_costo(config)
        if costo_actual < mejor_costo:
            mejor_costo = costo_actual
            mejor_configuracion = config

# Resultados
print("Mejor costo:", mejor_costo)
print("Mejor configuración:", mejor_configuracion)
print("Número de combinaciones generadas:", len(combinaciones))
print("Número de permutaciones generadas:", len(permutaciones))


Mejor costo: inf
Mejor configuración: None
Número de combinaciones generadas: 10
Número de permutaciones generadas: 10


### NOTA: Aquí tengo en cuenta sus indicaciones de feedback:

1. No descarto soluciones deterministas: he aquí el problema tratado con programacion dinamica



## (*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porqué crees que mejora el algoritmo por fuerza bruta

He elegido el Algoritmo de Programación Dinámica para el Problema de las Sesiones de Doblaje

Motivo:

Al irlo entendiendo y asimilando lo he relacionado mentalmente con el "Problema de la Mochila", que se puede resolver con programación dinámica, al igual que estos otros problemas que se pueden resolver con este algoritmo también se le parecen:

**Scheduling**: En nuestro problema, estamos programando las tomas en días, lo que se asemeja a programar tareas en un horario. 
**Problema de la mochila**: En nuestro problema, estamos seleccionando qué tomas se grabarán en cada día, lo que se asemeja a seleccionar qué objetos se incluyen en una mochila con un peso máximo. 
**Cutting stock problem**: En nuestro problema, estamos cortando tomas en días, lo que se asemeja a cortar piezas de material para minimizar el residuo. La programación dinámica se utiliza comúnmente en problemas de cutting stock para minimizar el residuo y maximizar el uso del material.

Además, me aseguré de que cumple las dos condiciones generales que deben cumplir los problemas para ser tratados por programacion dinámica:

1. Se puede alcanzar la solución a través de una secuencia de decisiones. 

2. La secuencia cumple con el principio de optimización.

En este problema, se tiene una capacidad limitada (la capacidad de grabar 6 tomas al día) y se deben seleccionar las tomas que se van a grabar en cada día de manera que se minimice el costo (el gasto por los desplazamientos de los actores).

El problema de la mochila es un problema clásico en la teoría de la complejidad computacional y se puede resolver utilizando técnicas de programación dinámica, como la tabla de Programación Dinámica (DP) utilizada en el código proporcionado.

En este problema, la capacidad limitada se refleja en la restricción de que se pueden grabar como máximo 6 tomas al día, y el costo se refleja en la cuota fija que cobran los actores por cada día que se desplazan para participar en una toma.

La solución al problema se obtiene construyendo una tabla DP que almacena el costo óptimo para grabar las tomas en cada día, y luego se reconstruye la lista óptima de tomas por día a partir de la tabla DP.

Subproblemas:

La clave está en descomponer el problema en subproblemas más pequeños. Cada subproblema consiste en encontrar el coste mínimo para un subconjunto de tomas desde la primera hasta la i-ésima toma.

Solape de Subproblemas: Los mismos subproblemas se resuelven múltiples veces. Por ejemplo, para calcular dp[10], necesitamos conocer los valores de dp[4], dp[5], ..., dp[9].

Subestructura Óptima: La solución óptima del problema completo se construye a partir de soluciones óptimas de sus subproblemas.

Definimos dp[i] como el coste mínimo para grabar las primeras i tomas.

-  Transición de Estados:

Para cada toma i, y cada posible número de tomas k que se pueden realizar en un día (donde k varía desde 1 hasta max_tomas_por_dia), calculamos el costo de asignar las tomas del día i-k al día i.

El costo se calcula con la función calcular_actores_por_dia, que determina los actores necesarios para un conjunto de tomas y devuelve su cantidad y la lista de actores involucrados.

-  Inicialización y Solución:

Inicializamos la tabla dp con un valor alto (np.inf) excepto dp[0], que se inicializa en 0.
Iteramos sobre todas las tomas y posibles días de grabación, actualizando la tabla dp y manteniendo un registro de las soluciones parciales en la tabla soluciones.


-  Justificación de la Mejora

El enfoque de programación dinámica majora la complejidad del algoritmo de fuerza bruta por las siguientes razones:

-  Reducción de Recomputaciones: al almacenar los resultados de subproblemas en la tabla dp, evitamos recalcular el costo de las mismas combinaciones de tomas y actores múltiples veces.

-  Optimización de Subproblemas: En lugar de evaluar todas las combinaciones posibles, el algoritmo de programación dinámica divide el problema en subproblemas más pequeños y optimiza cada subproblema de forma independiente, combinando sus resultados para obtener la solución final.

El algoritmo de programación dinámica es mucho más eficiente y manejable, lo que lo convierte en una mejora considerable respecto al enfoque de fuerza bruta.

In [8]:

# Cargar la matriz de participación desde el archivo
file_path = 'Datos problema.xlsx'
data = pd.read_excel(file_path)
participation_matrix = data.drop(columns=['Toma']).to_numpy()

num_tomas = participation_matrix.shape[0]
num_actores = participation_matrix.shape[1]
max_tomas_per_day = 6

# Función para calcular el coste de un día
def calcular_costo_dia(tomas):
    actores_participantes = np.any(participation_matrix[tomas, :], axis=0)
    return np.sum(actores_participantes)

# Inicialización de la tabla de DP
dp = [float('inf')] * (num_tomas + 1)
dp[0] = 0
particiones = [-1] * (num_tomas + 1)

# Rellenar la tabla de DP
for i in range(1, num_tomas + 1):
    for j in range(max(0, i - max_tomas_per_day), i):
        costo = calcular_costo_dia(slice(j, i))
        if dp[j] + costo < dp[i]:
            dp[i] = dp[j] + costo
            particiones[i] = j

# Reconstrucción de la lista óptima de tomas por día
dias = []
i = num_tomas
while i > 0:
    dias.append((particiones[i], i))
    i = particiones[i]

# Inversión de la lista para obtener el orden correcto
dias.reverse()

# Creación de la tabla de resultados y cálculo del costo total
resultado = pd.DataFrame({
    'Día': range(1, len(dias) + 1),
    'Tomas': [f"Tomas {inicio + 1} a {fin}" for inicio, fin in dias],
    'Costo': [calcular_costo_dia(slice(inicio, fin)) for inicio, fin in dias]
})

# Cálculo del costo total
costo_total = resultado['Costo'].sum()

# Mostrar el resultado
print(resultado)
print(f"Costo Total: {costo_total}")

   Día          Tomas  Costo
0    1    Tomas 1 a 6      7
1    2   Tomas 7 a 12      8
2    3  Tomas 13 a 18      8
3    4  Tomas 19 a 24      7
4    5  Tomas 25 a 30      8
Costo Total: 38


### (*)Calcula la complejidad del algoritmo

El orden de complejidad de este algoritmo es O(n*m), donde:

n es el número de tomas (num_tomas)

m es el número de actores (num_actores)

La razón es la siguiente:

El bucle forque rellena la tabla de DP itera num_tomas n veces, lo que da un orden de complejidad de O(n).

Dentro de ese bucle, hay otro bucle for que itera desde i - max_tomas_per_day hasta i, lo que da un orden de complejidad de O(m) en el peor caso.

El cálculo del costo de un día ( calcular_costo_dia) tiene un orden de complejidad de O(m), ya que itera sobre los actores.

La reconstrucción de la lista óptima de tomas por día itera num_tomas n veces, lo que da un orden de complejidad de O(n).

La creación de la tabla de resultados y el cálculo del costo total tienen un orden de complejidad de O(n), ya que itera sobre las filas de la tabla.

Por lo tanto, el orden de complejidad total es O(n*m), ya que el bucle for que rellena la tabla de DP es el que domina el orden de complejidad.


## Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

In [9]:

# Definir los tamaños del nuevo dataset
new_num_tomas = 100  # Aumentar el número de tomas a 100
new_num_actores = 20  # Aumentar el número de actores a 20

# Generar una nueva matriz de participación aleatoria
np.random.seed(42)  # Para reproducibilidad
new_participation_matrix = np.random.randint(2, size=(new_num_tomas, new_num_actores))

# Crear un nuevo DataFrame con las tomas y la nueva matriz de participación
new_data = pd.DataFrame(new_participation_matrix, columns=[f'Actor_{i+1}' for i in range(new_num_actores)])
new_data.insert(0, 'Toma', range(1, new_num_tomas + 1))

# Guardar el nuevo dataset en un archivo Excel
new_file_path = 'Nuevos_Datos_problema.xlsx'
new_data.to_excel(new_file_path, index=False)

# Mostrar una vista previa de los datos generados
print(new_data.head())

# Confirmar que el archivo ha sido guardado
print(f"Nuevo archivo guardado en: {new_file_path}")


   Toma  Actor_1  Actor_2  Actor_3  Actor_4  Actor_5  Actor_6  Actor_7  \
0     1        0        1        0        0        0        1        0   
1     2        1        0        1        1        1        1        1   
2     3        0        0        1        1        1        1        1   
3     4        0        0        0        0        0        1        1   
4     5        0        1        0        1        0        0        1   

   Actor_8  Actor_9  ...  Actor_11  Actor_12  Actor_13  Actor_14  Actor_15  \
0        0        0  ...         0         0         0         0         1   
1        1        1  ...         0         0         1         1         1   
2        0        1  ...         0         1         0         1         0   
3        0        1  ...         1         1         0         1         0   
4        0        1  ...         1         1         1         1         1   

   Actor_16  Actor_17  Actor_18  Actor_19  Actor_20  
0         0         1         1 

## Aplica el algoritmo al juego de datos generado

In [10]:

# Cargar la matriz de participación desde el archivo
file_path = 'Nuevos_Datos_problema.xlsx'
data = pd.read_excel(file_path)
participation_matrix = data.drop(columns=['Toma']).to_numpy()

num_tomas = participation_matrix.shape[0]
num_actores = participation_matrix.shape[1]
max_tomas_per_day = 6

# Función para calcular el coste de un día
def calcular_costo_dia(tomas):
    actores_participantes = np.any(participation_matrix[tomas, :], axis=0)
    return np.sum(actores_participantes)

# Inicialización de la tabla de DP
dp = [float('inf')] * (num_tomas + 1)
dp[0] = 0
particiones = [-1] * (num_tomas + 1)

# Rellenar la tabla de DP
for i in range(1, num_tomas + 1):
    for j in range(max(0, i - max_tomas_per_day), i):
        costo = calcular_costo_dia(slice(j, i))
        if dp[j] + costo < dp[i]:
            dp[i] = dp[j] + costo
            particiones[i] = j

# Reconstrucción de la lista óptima de tomas por día
dias = []
i = num_tomas
while i > 0:
    dias.append((particiones[i], i))
    i = particiones[i]

# Inversión de la lista para obtener el orden correcto
dias.reverse()

# Creación de la tabla de resultados y cálculo del costo total
resultado = pd.DataFrame({
    'Día': range(1, len(dias) + 1),
    'Tomas': [f"Tomas {inicio + 1} a {fin}" for inicio, fin in dias],
    'Costo': [calcular_costo_dia(slice(inicio, fin)) for inicio, fin in dias]
})

# Cálculo del costo total
costo_total = resultado['Costo'].sum()

# Mostrar el resultado
print(resultado)
print(f"Costo Total: {costo_total}")

    Día           Tomas  Costo
0     1     Tomas 1 a 6     20
1     2    Tomas 7 a 10     15
2     3   Tomas 11 a 16     20
3     4   Tomas 17 a 22     20
4     5   Tomas 23 a 28     20
5     6   Tomas 29 a 34     20
6     7   Tomas 35 a 40     20
7     8   Tomas 41 a 46     19
8     9   Tomas 47 a 52     20
9    10   Tomas 53 a 58     20
10   11   Tomas 59 a 64     20
11   12   Tomas 65 a 70     19
12   13   Tomas 71 a 76     19
13   14   Tomas 77 a 82     20
14   15   Tomas 83 a 88     19
15   16   Tomas 89 a 94     20
16   17  Tomas 95 a 100     19
Costo Total: 330


## Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

He aplicado lo visto en clase.

## Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

##  NOTA: Aquí he tenido en cuenta esto otro que me decia en su feedback: 

"Te has decantado por un metaheuristico, lo cual me parece bien. Eso si, me faltan muchas explicaciones en la entrega (ej. por que descartas soluciones deterministas, 
o como vas a comparar la calidad de las distintas soluciones o porque crees que el incorporar una solucion tabu te empeora)."


Pues bien, en cuanto los datos crecen un poco nos damos cuenta de que el algoritmo de programación dinámica se nos va a quedar muy corto, además no es capaz de refinar soluciones.

Pero al implementar el algoritmo TABU con el juego de soluciones pequeño me daba una solución mala, peor que la solución del algoritmo de programación dinámica, y eso me despistó.
Pero me di cuenta estudiando y haciendolo que cuando despliega todo su potencial es cuando aumentamos el tamaño de los datos, como debe ser y 
para eso estan destinados los algoritmos metaheurísitcos en general.

Y tambien como el funcionamiento general de las técnicas metaheurísticas consiste en mantener almacenada durante el proceso una solución o estado de referencia, que es el mejor encontrado hasta el momento a través de las iteraciones, pues pensé que en este problema nuestro de las tomas nos podria venir bien en concreto el ALGORITMO TABÚ ya que el propósito de la lista tabú es evitar volver a soluciones encontradas recientemente para evitar los ciclos en el proceso. 
La mejor solución hasta el momento se guarda para ser considera la solución final.

El tamaño de la lista tabú es el que controla la memoria del proceso, de tal manera que, para tamaños pequeños, el proceso se concentra en espacios de soluciones pequeños. Sin embargo, para tamaños grandes de la lista, el proceso explora espacios más grandes de soluciones.

Es que me parece que se ajusta muy bien a nuestro problema.

Para comparar la calidad de las soluciones:

-  evaluando el valor objetivo de cada solución.
  
- por la cantidad de diferentes soluciones encontradas.
  
- si van convergiendo de las soluciones.



Ahora voy a implementar el algoritmo TABU como solucion general.


GENERAR SOLUCION INICIAL

Para asegurar que la solución inicial sea siempre la misma y pueda revisar el resultado, 

tanto a la función generar_solucion_inicial como a generar vecino le he pasado un valor fijo de semilla en la generación aleatoria.

llamar a random.seed(semilla)

ya lo cambiaré luego cuando continue el ejercicio

GENERAR VECINO

Genera una solución vecina intercambiando tomas entre dos días seleccionados aleatoriamente. Con semilla por ahora


PASOS:

- Inicialización

Genera una solución inicial y calcula su costo.
Inicializa la lista Tabú y establece la mejor solución y su costo.

- Iteraciones

Para cada iteración, genera una solución vecina.
Calcula el costo de la solución vecina.
Si la solución vecina no está en la lista Tabú y es mejor que la solución actual o la mejor solución encontrada, 

actualiza la solución actual y, si corresponde, la mejor solución.

Añade la solución vecina a la lista Tabú y, si la lista excede el tamaño máximo, elimina la solución más antigua.


In [36]:
### Leer el archivo Excel en un DataFrame
file_path = 'Nuevos_Datos_problema.xlsx'

df = pd.read_excel(file_path)  

datos_tomas_tabu = df


# Evalúa cuántos actores diferentes están presentes en una lista de tomas.

def evaluar_lista(tomas, datos_tomas_tabu):
    num_actores = 10
    actores_presentes = set()
    for toma in tomas:
        toma_idx = datos_tomas_tabu.index[datos_toma_tabu['Toma'] == toma].tolist()[0]
        for actor in range(num_actores):
            if datos_tomas_tabu.iloc[toma_idx, actor + 1] == 1:
                actores_presentes.add(actor)
    return len(actores_presentes)


#esta ya la tenia del algoritmo de programacion dinamica
#Calcula el número de actores presentes cada día, dada una distribución de tomas.

def calcular_actores_por_dia(dias_tomas, datos_tomas_tabu):
    num_actores = 10
    actores_por_dia = {}
    
    for dia, tomas in dias_tomas.items():
        actores_presentes = set()
        for toma in tomas:
            toma_idx = datos_tomas_tabu.index[datos_tomas_tabu['Toma'] == toma].tolist()[0]
            for actor in range(num_actores):
                if datos_tomas_tabu.iloc[toma_idx, actor + 1] == 1:
                    actores_presentes.add(actor)
        actores_por_dia[dia] = len(actores_presentes)
    
    return actores_por_dia

# Calcula el costo total (número total de actores) sumando los actores diferentes presentes cada día.
def calcular_costo(dias_tomas, datos_tomas_tabu):
    actores_por_dia = calcular_actores_por_dia(dias_tomas, datos_tomas_tabu)
    return sum(actores_por_dia.values())

    
def generar_solucion_inicial(datos_tomas_tabu, semillai=42):
    random.seed(semillai)  # Establece la semilla para la generación aleatoria
    tomas_restantes = list(datos_tomas_tabu['Toma'])
    dias_tomas = {}
    dia = 1
    
    while tomas_restantes:
        if len(tomas_restantes) <= 6:
            dias_tomas[dia] = tomas_restantes
            break
        else:
            lista_actual = random.sample(tomas_restantes, 6)
            dias_tomas[dia] = lista_actual
            tomas_restantes = [toma for toma in tomas_restantes if toma not in lista_actual]
        dia += 1
    
    return dias_tomas

def generar_vecino(solucion_actual, semillav=123):
    random.seed(semillav)  # Establece la semilla para la generación aleatoria
    vecino = solucion_actual.copy()
    dias = list(vecino.keys())
    dia1, dia2 = random.sample(dias, 2)
    
    if vecino[dia1] and vecino[dia2]:
        toma1 = random.choice(vecino[dia1])
        toma2 = random.choice(vecino[dia2])
        vecino[dia1].remove(toma1)
        vecino[dia1].append(toma2)
        vecino[dia2].remove(toma2)
        vecino[dia2].append(toma1)
    
    return vecino

def tabu_search(datos_tomas_tabu, iteraciones, lista_tabu_tamano):
    solucion_actual = generar_solucion_inicial(datos_tomas_tabu)
    costo_actual = calcular_costo(solucion_actual, datos_tomas_tabu)
    mejor_solucion = solucion_actual
    mejor_costo = costo_actual
    lista_tabu = []

    for _ in range(iteraciones):
        vecino = generar_vecino(solucion_actual)
        costo_vecino = calcular_costo(vecino, datos_tomas_tabu)
        
        if vecino not in lista_tabu and (costo_vecino < costo_actual or costo_vecino < mejor_costo):
            solucion_actual = vecino
            costo_actual = costo_vecino
            if costo_vecino < mejor_costo:
                mejor_solucion = vecino
                mejor_costo = costo_vecino
            
            lista_tabu.append(vecino)
            if len(lista_tabu) > lista_tabu_tamano:
                lista_tabu.pop(0)
    
    return mejor_solucion, mejor_costo

# Parámetros de la Búsqueda Tabú
iteraciones = 1000
lista_tabu_tamano = 50

# Ejecutar la Búsqueda Tabú
mejor_solucion, mejor_costo = tabu_search(datos_tomas_tabu, iteraciones, lista_tabu_tamano)

# Calcular actores diferentes que trabajan cada día
actores_por_dia = calcular_actores_por_dia(mejor_solucion, datos_tomas_tabu)
total_sueldos = sum(actores_por_dia.values())

# Imprimir los resultados
for dia, tomas in mejor_solucion.items():
    print(f"Día {dia}: {tomas}")
print(f"Actores diferentes que trabajan cada día: {actores_por_dia}")
print(f"Total de sueldos a pagar: {total_sueldos}")

Día 1: [82, 15, 4, 95, 36, 32]
Día 2: [20, 16, 92, 74, 13, 21]
Día 3: [86, 63, 6, 5, 14, 35]
Día 4: [42, 79, 96, 7, 88, 38]
Día 5: [91, 70, 44, 75, 51, 1]
Día 6: [33, 77, 62, 54, 30, 45]
Día 7: [68, 12, 11, 43, 99, 40]
Día 8: [94, 47, 67, 29, 89, 8]
Día 9: [87, 59, 66, 53, 18, 31]
Día 10: [78, 48, 85, 84, 56, 80]
Día 11: [28, 17, 9, 37, 49, 19]
Día 12: [52, 25, 72, 27, 100, 64]
Día 13: [97, 50, 24, 98, 90, 26]
Día 14: [93, 46, 10, 34, 73, 41]
Día 15: [55, 58, 57, 39, 65, 60]
Día 16: [23, 69, 2, 3, 76, 22]
Día 17: [61, 71, 81, 83]
Actores diferentes que trabajan cada día: {1: 10, 2: 10, 3: 10, 4: 10, 5: 10, 6: 10, 7: 10, 8: 10, 9: 10, 10: 10, 11: 10, 12: 10, 13: 10, 14: 10, 15: 10, 16: 10, 17: 10}
Total de sueldos a pagar: 170


He generado distintos juegos de datos, cada vez mas grandes, no incluyo aquí todas las pruebas. 

Creo que de verdad lo hace francamente bien y en muy buen tiempo, no se me cuelga. y los resultados que salen son muy buenos.

Sinceramente, creo que he acertado y me ha gustado mucho hacerlo funcionar y ver qué sacaba, muy curioso y entiendes realmente la estrategia.